In [1]:
import numpy as np 
import pandas as pd 
from scipy import stats
import calendar
import matplotlib as plt
import category_encoders as ce
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import svm

In [2]:
pub_df = pd.read_csv('data/ks-projects-201801.csv')
ins_df = pd.read_csv('data/insurance_event.csv')

In [18]:
new_col = []
for row in pub_df['category']:
    new_col.append(row)
for row in ins_df['Event_Code'].head(1000):
    new_col.append(row)    
new_col=set(new_col)
new_col=list(new_col)

In [25]:
col_df = pd.DataFrame({'Test':new_col})
col_df.Test.unique()

array(['6989', '3325', '1817', '9920', '3486', '8007', '8470', 'V438',
       'Puzzles', '2425', 'Movie Theaters', 'Romance', '4109', '3328',
       '1104', '3241', 'Live Games', '3059', '9703', 'E928',
       'Science Fiction', 'J250', '6260', '9641', '3510', 'Kids', '3004',
       '2617', '3131', 'Performance Art', '3082', 'Embroidery', '7943',
       '2605', '7729', '9326', '2990', 'Sound', '3469', '2961',
       'Young Adult', 'Painting', '3464', '9711', '3577', 'T101', '8267',
       '7307', 'Fine Art', '7474', '3610', '3517', 'People', '1967',
       'V436', 'V048', 'Pottery', '6489', '7159', '8005', '7879', '2648',
       '4770', 'Fiction', 'Musical', 'V103', '4019', 'Crochet', '2178',
       '2235', '3466', 'Product Design', '3462', '2508', '5984', '8708',
       'E885', '7962', '6494', '1579', 'Mobile Games', 'V854', '2624',
       'G855', 'Publishing', '6180', '2803', 'J078', '2833', '7221',
       '2229', 'Nature', 'Spaces', 'J705', 'J348', '3266', 'Webseries',
       'Perfo

In [26]:
def get_no_of_collisions(df,column,n):
    fh = FeatureHasher(n_features=n, input_type='string')
    hashed_features = fh.fit_transform(df[column].drop_duplicates()).toarray()
    fh_df = pd.DataFrame(hashed_features)
    g = fh_df.groupby(fh_df.columns.tolist())
    return len(fh_df) - g.ngroups

In [29]:
n_limit = {2,4,6,8,10,16,32,64,128,256,512,1024,2048}
for n in n_limit:
    print("Collision for column {} when n_features = {} is : {} ".format('Test',n,get_no_of_collisions(col_df,'Test',n)))

Collision for column Test when n_features = 32 is : 206 
Collision for column Test when n_features = 64 is : 206 
Collision for column Test when n_features = 2 is : 717 
Collision for column Test when n_features = 128 is : 193 
Collision for column Test when n_features = 4 is : 605 
Collision for column Test when n_features = 256 is : 193 
Collision for column Test when n_features = 6 is : 466 
Collision for column Test when n_features = 512 is : 193 
Collision for column Test when n_features = 8 is : 463 
Collision for column Test when n_features = 1024 is : 193 
Collision for column Test when n_features = 10 is : 416 
Collision for column Test when n_features = 2048 is : 193 
Collision for column Test when n_features = 16 is : 288 
